In [1]:
import twitterGraph as tg
import numpy as np
import random as rand
graph = tg.twitterGraph()

In [2]:
graph.numUsers

41652229

In [6]:
graph.getFollowers(graph.getRandomUser())

array([    9367,   813286, 10886522, 17473498, 18036928, 19165424,
       21549658, 21924949, 23330108, 24124991, 24341946, 26918419,
       27529678, 27998872, 28175389, 30115654, 32779133, 39017517,
       39048911, 39053817, 40822836, 57949579, 57976136, 58649072],
      dtype=uint32)